# Языковое моделирование
Курс NLP, лабораторная 1.  
Осень 2015.

В данном задании вы реализуете:

- Add-one smoothing
- Stupid backoff
- Interpolation smoothing
- EM-algorithm
- Kneser-Ney smoothing

Вы примените это к:

- Language recognition problem

**Подпись**: *Ластовичек Дмитрий Владимировичк - АД - 2 курс*

Цель языкового моделирования заключается в том, чтобы присвоить некоторые вероятности предложениям. Возникает закономерный вопрос, а зачем нам это надо? Например, в задачах _машинного перевода_ частенько нужно среди нескольких предложений выбирать наиболее вероятный перевод (который является естественным для человеческого глаза). Также это чрезвычайно полезно в задаче _исправления опечаток_ и _распозновании речи_.

Наша задача состоит в подсчете вероятности $P(W) = P(w_1, \dots, w_n)$ или $P(w_n \mid w_1, \dots, w_{n-1})$. Модель, умеющая вычислять хотя бы одну из этих двух вероятностей, называется **язковой моделью** (LM от Language Model).

Пришло время вспомнить _цепное правило_ (chain rule) $P(X_1, \dots, X_n) = P(X_1)P(X_2 \mid X_1)\dots P(X_n \mid X_1, \dots, X_{n-1})$. Также мы знаем, что

$$
    P(X_n \mid X_1, \dots, X_{n-1}) = \frac{P(X_1, \dots, X_n)}{P(X_1, \dots, X_{n-1})},
$$

следовательно, для того чтобы оценить $P(X_n \mid X_1, \dots, X_{n-1})$ нужно посчитать $P(X_1, \dots, X_n)$ и $P(X_1, \dots, X_{n-1})$. Но эти вероятности будут чрезвычайно малы, если мы возьмем большое $n$, так множество предложений из $n$ слов растет экспоненциально. Для упрощения ситуации применим **марковское предположение**: $P(X_n \mid X_1, \dots, X_{n-1}) = P(X_n \mid X_{n - k + 1}, \dots, X_{n-1})$ для некоторого фиксированного (небольшого) $k$. Это предположение интуитивно ясно и говорит нам о том, что $X_{n}$ не зависит от $X_{1}, \dots, X_{n - k}$, то есть на следующее слово влияет лишь контекст из предыдущих $k - 1$ слова. Таким образом, мы получаем финальную вероятность:

$$
    P(w_1, \dots, w_n) = \prod_i P(w_i \mid w_{i-k+1}, \dots, w_{i - 1}).
$$

Далее для краткости будем обозначать $w_{i-k}^i := w_{i-k}, \dots, w_{i}$.

## Хранилище n-грам

Пришло время написать класс для хранения n-грамм. Следуйте комментариям, чтобы написать NGramStorage с удобным интерфейсом.

In [160]:
import re
import math
import random
from collections import Counter
from collections import defaultdict
import numpy as np
from sklearn.cross_validation import train_test_split
import os
from os import walk
from os import path
import sklearn
import sklearn.metrics
import csv

In [2]:
class NGramStorage:
    """Storage for ngrams' frequencies.
    
    Args:
        sents (list[list[str]]): List of sentences from which ngram
            frequencies are extracted.
        max_n (int): Upper bound of the length of ngrams.
            For instance if max_n = 2, then storage will store
            0, 1, 2-grams.
            
    Attributes:
        max_n (Readonly(int)): Upper bound of the length of ngrams.
    """
        
    def __init__(self, sents=[], max_n=0):
        self.__max_n = max_n
        self.__ngrams = {i: Counter() for i in range(self.__max_n + 1)}
        # self._ngrams[K] should have the following interface:
        # self._ngrams[K][(w_1, ..., w_K)] = number of times w_1, ..., w_K occured in words
        # self._ngrams[0][()] = number of all words
        for ngram_size in self.__ngrams:
            for sentence in sents:
                sentence_length = len(sentence)
                for word_number in range(sentence_length - ngram_size):
                    self.__ngrams[ngram_size][tuple(sentence[word_number:(word_number + ngram_size)])] += 1
        
    def add_unk_token(self):
        """Add UNK token to 1-grams."""
        # In order to avoid zero probabilites 
        if self.__max_n == 0 or u'UNK' in self.__ngrams[1]:
            return
        self.__ngrams[0][()] += 1
        self.__ngrams[1][(u'UNK',)] = 1
        
    @property
    def max_n(self):
        """Get max_n"""
        return self.__max_n
        
    def __getitem__(self, k):
        """Get dictionary of k-gram frequencies.
        
        Args:
            k (int): length of returning ngrams' frequencies.
            
        Returns:
            Dictionary (in fact Counter) of k-gram frequencies.
        """
        # Cheking the input
        if not isinstance(k, int):
            raise TypeError('k (length of ngrams) must be an integer!')
        if k > self.__max_n:
            raise ValueError('k (length of ngrams) must be less or equal to the maximal length!')
        return self.__ngrams[k]
    
    def __call__(self, ngram):
        """Return frequency of a given ngram.
        
        Args:
            ngram (tuple): ngram for which frequency should be computed.
            
        Returns:
            Frequency (int) of a given ngram.
        """
        # Cheking the input
        if not isinstance(ngram, tuple):
            raise TypeError('ngram must be a tuple!')
        if len(ngram) > self.__max_n:
            raise ValueError('length of ngram must be less or equal to the maximal length!')
        if len(ngram) == 1 and ngram not in self.__ngrams[1]:
            return self.__ngrams[1][(u'UNK', )]
        return self.__ngrams[len(ngram)][ngram]

Давайте скачаем корпус и запустим на нем наши эксперименты.

In [3]:
import nltk
# Uncomment next row and download brown corpus
# nltk.download()
from nltk.corpus import brown

In [4]:
all_sents = list(brown.sents())
random.shuffle(all_sents)
print('Number of all sentences = {}'.format(len(all_sents)))
train_sents = all_sents[:int(0.8 * len(all_sents))]
test_sents = all_sents[int(0.8 * len(all_sents)):]
print('Number of train sentences = {}'.format(len(train_sents)))
print('Number of test sentences = {}'.format(len(test_sents)))

Number of all sentences = 57340
Number of train sentences = 45872
Number of test sentences = 11468


In [188]:
# Create storage of 0, 1, 2, 3-grams
storage = NGramStorage(train_sents, 3)

In [6]:
# It's time to test your code
print(storage(('to', 'be')))
print(storage(('or',)))
print(storage(('not', 'to', 'be')))
print(storage(('Muammar',)))
print(storage(()))

1360
3266
30
0
929972


## Оценка качества

Определим **перплексию**:

$$
    {\rm PP}(w_1, \dots, w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_i P(w_i \mid w_{i - k}, \dots, w_{i - 1})\right)^{-\frac1N},
$$

кстати, из этой формулы видно, что задача по минимизации перплексии равносильна задаче по максимизации вероятности.

Реализуйте функцию по подсчету перплексии. Стоит отметить, что перплексия по корпусу $-$ это не то же самое, что и усредненная перплексия по предложениям. Перплексия по корпусу равна произведению вероятностей **всех** предложений в степени $-\frac1N$, где $N -$ суммарная длина всех предложений.

In [239]:
def perplexity(estimator, sents):
    '''Estimate perplexity of the sequence of words using prob_estimator.'''
    # Avoid log(0) by replacing zero by 10 ** (-50).
    perp = 0.
    N = 0
    for sentence in sents:
        sentence_prob = estimator.prob(sentence)
        N += len(sentence)
        if sentence_prob < 10 ** (-50):
            sentence_prob = 10 ** (-50)
        perp += math.log(sentence_prob)
    perp *= -1. / N
    perp = math.exp(perp)
    
    return perp

## Оценка вероятностей n-грам

Первый и простейший способ оценки вероятностей N-грам следующий:

$$
    \hat P_{S}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N)}{c(w_1^{N-1})}.
$$

где $c(w_1^N)$ — это число последовательностей $w_1, \dots, w_N$ в корпусе, $S$ символизирует Straightforward. Что-ж, пора это реализовать.

In [7]:
class StraightforwardProbabilityEstimator:
    """Class for simplest probability estimations of type P(word | context).
    
    P(word | context) = c(context + word) / c(context), where
    c(sequence) - number of occurances of the sequence in the corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
    """
    
    def __init__(self, storage):
        self.__storage = storage
        # Adding UNK token to avoid zero probabilities
        self.__storage.add_unk_token()
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if self.__storage.max_n == 1:
            return ()
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        #if not isinstance(word, str):
        #    print type(word)
        #    raise TypeError('word must be a string!')
        #if not isinstance(context, tuple):
        #    raise TypeError('word must be a string!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        # Avoiding 0 / 0.
        if context_counts == 0:
            return 0.
        return 1. * phrase_counts / context_counts
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [83]:
# Initialize estimator
simple_estimator = StraightforwardProbabilityEstimator(storage)

# Estimating perplexity
print('Simple estimator perplexity = {}'.format(perplexity(simple_estimator, test_sents)))
print(simple_estimator.prob('To be'.split()))
print(simple_estimator.prob('To be or not to be'.split()))

Simple estimator perplexity = 285.666013616
1.82808082698e-05
0.0


Посчитаем перплексию униграмной модели.

In [84]:
uni_storage = NGramStorage(train_sents, 1)
uni_simple_estimator = StraightforwardProbabilityEstimator(uni_storage)
print('Simple estimator perplexity = {}'.format(perplexity(uni_simple_estimator, test_sents)))

Simple estimator perplexity = 126.793640123


___
<font color='red'>Ответьте на следующие вопросы (внутри ipython ноутбука):</font>

**Q:** Какие выводы можно сделать? Почему $P(\text{To be or not to be}) = 0$, хотя мы и добавили UNK токен?  
**A:** Потому что мы используем слишком примитивную формулу для оценки вероятности. Если какой-то n-gram не встречается - то вероятность равна 0

**Q:** Почему перплексия униграмной модели меньше, чем триграмной?  
**A:** На самом деле должно быть наоборот, предложений с вероятностью <10e-50 больше в униграмной модели, но так как в формуле перплексии мы корректируем вероятность таких предложений, то она возрастает. Если все предложения с малой вероятностью просто пропустить в вычислении перплексии, то у униграмной модели она будет больше.
___

## Add-one smoothing

Простейший вид сглаживания — **сглаживание Лапласа**. Каким-то образом мы хотим избавиться от нулевых вероятностей. Наиболее простой алгоритм борьбы с этой проблемой следующий: для оценки $P(w_{N} \mid w_1^{N - 1})$ мы будем использовать формулу:

$$
    \hat P_{AOS}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N) + \delta}{c(w_1^{N-1}) + \delta V},
$$

где $V$ — это размер словаря, а $\delta$ — некоторая фиксированная константа.

Теперь пришло время снова пописать код. Реализуйте класс, симулирующий сглаживание Лапласа. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [8]:
class LaplaceProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = (c(context + word) + delta) / (c(context) + delta * V), where
    c(sequence) - number of occurances of the sequence in the corpus,
    delta - some constant,
    V - number of different words in corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): Smoothing parameter.
    """
    
    def __init__(self, storage, delta=1.):
        self.__storage = storage
        self.__delta = delta
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        #if not isinstance(word, str):
        #    raise TypeError('word must be a string!')
        #if not isinstance(context, tuple):
        #    raise TypeError('context must be a tuple!')
            
        ### YOUR CODE HERE
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        # Avoiding 0 / 0.
        if context_counts == 0:
            return 0.
        voc_size = len(context)
        if voc_size == 0:
            voc_size = 1
        return 1. * (phrase_counts + self.__delta) / (context_counts + self.__delta * len(self.__storage[voc_size]))
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

Подберите наилучший параметр $\delta$ для данного корпуса.

In [86]:
# Try to find out best delta parametr. We will not provide you any strater code.
### YOUR CODE HERE
best_delta = 1.
min_perp = 10000
for delta in 10.**(-np.arange(10)):
    laplace_estimator = LaplaceProbabilityEstimator(storage, delta)
    perp = perplexity(laplace_estimator, test_sents)
    if (perp < min_perp):
        min_perp = perp
        best_delta = delta
print 'Best delta:', best_delta
        ### END YOUR CODE

# Initialize estimator
laplace_estimator = LaplaceProbabilityEstimator(storage, best_delta)

# Let's make some estimations
print('Laplace estimator perplexity = {}'.format(perplexity(laplace_estimator, test_sents)))
print(laplace_estimator.prob('To be'.split()))

Best delta: 0.0001
Laplace estimator perplexity = 231.45505721
1.80203529075e-05


## Stupid backoff

Идея **тупого откатывания** невероятно проста (на то оно и тупое!). Если у нас есть достаточно информцаии для подсчета вероятности $k$-грам, тогда будем использовать $k$-грамы. Если эта вероятность крайне мала, то будем использовать вероятности $(k-1)$-грам с некоторым множителем, например, $0.4$, и так далее. К сожалению, в данном случае мы получим не вероятностное распределение, но в большинстве задач это не принципиально важно.

Реализуйте класс, симулирующий сглаживание тупым откатыванием. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [9]:
class StupidBackoffProbabilityEstimator:
    """Class for stupid backoff probability estimations.
    
    P(word | context) =
        P'(word | context),                  if  P'(word | context) > 0;
        P'(word | context[1:]) * multiplier, if  P'(word | context) == 0
                                             and P'(word | context[1:]) > 0;
        ...
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        multiplier (float): Multiplier which is used for probability estimations.
    """
    
    def __init__(self, base_estimator, multiplier=0.1):
        self.__base_estimator = base_estimator
        self.__mult = multiplier
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        
        ### YOUR CODE HERE
        prob = self.__base_estimator(word, context)
        if prob == 0:
            return self.__base_estimator(word, context[1:])
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [88]:
# Initialize estimator
sbackoff_estimator = StupidBackoffProbabilityEstimator(simple_estimator, .4)

# Let's make some estimations
print('Stupid backoff estimator perplexity = {}'.format(perplexity(sbackoff_estimator, test_sents)))
print(laplace_estimator.prob('To be'.split()))

Stupid backoff estimator perplexity = 267.749178065
1.80203529075e-05


___
<font color='red'>Ответьте на следующие вопросы (внутри ipython ноутбука):</font>

**Q:** Почему бессмысленно измерять перплексию в случае **Stupid backoff**?  
**A:** Потому что на менее вероятном корпусе может возникнуть больше нулевых вероятностей и данное сглаживание его улучшит по сравнению с более хорошим корпусом в котором будет меньше нулевых вероятностей
___

## Interpolation smoothing

В данном случае идея сглаживания посредством **интерполяции** также крайне проста. Пусть у нас есть $N$-грамная модель. Заведем вектор $\bar\lambda = (\lambda_1, \dots, \lambda_N)$, такой, что $\sum_i\lambda_i = 1$ и $\lambda_i \geq 0$. Ну а далее просто постулируем, что

$$
    \hat P_{IS}(w_{N} \mid w_1^{N-1}) = \sum_{i=1}^N \lambda_i \hat P_{S}(w_N \mid w_{N-i+1}^{N-1}).
$$

Придумайте, как обойтись одним вектором $\bar\lambda$. Казалось бы, их нужно несколько, ибо наша модель должна уметь считать вероятности $P(w_3 \mid w_1, w_2)$, а иногда $P(w_2 \mid w_1)$. Если мы тупо обрубим сумму, то у нас уже не будет вероятностное распределение, что, конечно же, плохо.

In [10]:
class InterpolationProbabilityEstimator:
    """Class for interpolation probability estimations.
    
    P(word | context) =
        lambda_N * P'(word | context) +
        lambda_{N-1} * P'(word | context[1:]) +
        ... +
        lambda_1 * P'(word)
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        lambdas (np.array[float]): Lambdas which are used for probability estimations.
    """
    
    def __init__(self, base_estimator, lambdas):
        self.lambdas = lambdas
        self.__base_estimator = base_estimator
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        if len(context) == 0:
            return  self.__base_estimator(word, context)
        
        lambda_corrected = self.lambdas[-len(context):] / sum(self.lambdas[-len(context):])
        ### YOUR CODE HERE
        prob = 0.
        for i in range(len(lambda_corrected)):
            context_new = context[-i:]
            if i == 0:
                context_new = ()
            prob += lambda_corrected[i] * self.__base_estimator(word, context_new)
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [91]:
import numpy as np

# Initialize estimator
interpol_estimator = InterpolationProbabilityEstimator(simple_estimator, np.array([0.2, 0.2, 0.6]))

# Let's make some estimations
print('Interpolation estimator perplexity = {}'.format(perplexity(interpol_estimator, test_sents)))
print(interpol_estimator.prob('To be'.split()))

Interpolation estimator perplexity = 118.922046085
2.05578319865e-06


Остается один вопрос: как подбирать $\bar\lambda$? Для этого обычно применяется EM-алгоритм. Сразу опишем лишь готовую формулу. Пусть $\bar\lambda^t$ соответствует набору коэффициентов на $t$-м шаге. $\bar\lambda^0$ задаем произвольно.

* **E-step**:
$$
    \hat\lambda_j^t = \sum_{w_1^N} \frac{\lambda_j^t \cdot \hat P_{S}(w_N \mid w_{N-j+1}^{N-1})}{\sum_{i=1}^N \lambda_i^t \hat P_{S}(w_N \mid w_{N-i+1}^{N-1})}.
$$
* **M-step**:
$$
    \lambda_j^{t+1} = \frac{\hat\lambda_j^t}{\sum_{i=1}^N \hat\lambda_i^t}.
$$

Формулы выписаны, то бишь самое сложное сделано. Вам остается это лишь реализовать.

In [225]:
def E_step(test_storage, s_estimator, i_estimator):
    ### YOUR CODE HERE
    N = len(i_estimator.lambdas)
    estimated_lambdas = np.zeros(N)
    
    for ngram in test_storage[N]:
        ngram_lambdas = np.zeros(N)
        word = ngram[-1:][0]
        context = ngram[:-1]
        for j in range(N):
            context_new = context[-j:]
            if j == 0:
                context_new = ()
            ngram_lambdas[j] += i_estimator.lambdas[j] * s_estimator(word, context_new)
        ngram_lambdas /= np.sum(ngram_lambdas)
        estimated_lambdas += ngram_lambdas
    ### END YOUR CODE
    
    return estimated_lambdas

def M_step(estimated_lambdas):
    ### YOUR CODE HERE
    new_lambdas = estimated_lambdas / sum(estimated_lambdas)
    ### END YOUR CODE
    
    return new_lambdas

def EM_algorithm(test_storage, s_estimator, i_estimator, epsilon=0.03):
    ### YOUR CODE HERE
    lambdas_old = i_estimator.lambdas
    step_number = 0
    while np.max(i_estimator.lambdas - lambdas_old) > 0.03 or step_number == 0:
        step_number += 1
        lambdas_e = E_step(test_storage, s_estimator, i_estimator)
        lambdas_old = i_estimator.lambdas
        i_estimator.lambdas = M_step(lambdas_e)
    ### END YOUR CODE
    print "Number of steps=", step_number
    return i_estimator.lambdas

In [228]:
# Separate train into train and test
train_set = train_sents[:int(0.6 * len(train_sents))]
test_set = train_sents[int(0.6 * len(train_sents)):]
MAX_N = 3
train_storage = NGramStorage(train_set, MAX_N)
test_storage = NGramStorage(test_set, MAX_N)
s_estimator = StraightforwardProbabilityEstimator(train_storage)

# Make starting assumption
starting_lambdas = np.array([0.33, 0.33, 0.34])
i_estimator = InterpolationProbabilityEstimator(s_estimator, starting_lambdas)

In [229]:
# It can take some time
best_lambdas = EM_algorithm(test_storage, s_estimator, i_estimator)
print "best lambdas=", best_lambdas

Number of steps= 3
best lambdas= [ 0.57765308  0.38167713  0.04066979]


In [230]:
# Initialize estimator
i_estimator = InterpolationProbabilityEstimator(simple_estimator, best_lambdas)

# Let's make some estimations
print('Interpolation estimator perplexity = {}'.format(perplexity(i_estimator, test_sents)))
print(i_estimator.prob('To be'.split()))

Interpolation estimator perplexity = 113.529942857
2.05578319865e-06


## Kneser-Ney smoothing

Идея данного сглаживания заключается в том, что словам, которые участвуют в большом количестве контекстов, присваиваются большие вероятности, а те, которые исползуются в паре-тройке контекстов получают маленькие вероятности. Авторы данного сглаживания формализовали это следующим образом. Введем обозначения

$$
    N_{1+}(\cdot w_2) := \left|\{w_1 : c(w_1, w_2) > 0\}\right|,
$$
$$
    N_{1+}(\cdot \cdot) := \sum_{w_2} N_{1+}(\cdot w_2),
$$
$$
    \hat P_{KN} (w_1) = \frac{{\rm max}\{N_{1+}(\cdot w_1)-\delta,0\}}{N_{1+}(\cdot \cdot)} + \frac{\delta}{|V|}.
$$

Далее мы используем реккурентное соотношение

$$
    \hat P_{KN}(w_{N} \mid w_1^{N-1}) = \frac{{\rm max}\{c(w_1^N) - \delta, 0\}}{\sum_{w_N}c(w_1^{N-1}w_N)} + \frac{\delta}{\sum_{w_N}c(w_1^{N-1}w_N)}N_{1+}(w_1^{N-1}\cdot)\hat P_{KN}(w_N \mid w_2^{N-1}).
$$

Для вас дело за малым — реализовать это.

In [257]:
class KneserNeyProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = ...
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): KneserNey parameter.
    """
    
    def __init__(self, storage, delta=1.):
        self.__storage = storage
        self.__delta = delta
        
        self.__n1_plus = self.__calc_n1_plus()
        self.__voc_size = len(self.__storage[1])
        
        self.__n1_left = Counter()
        for word1, word2 in self.__storage[2].keys():
            self.__n1_left[word2] += 1
            
        self.__n1_right_count = {i: Counter() for i in range(1,self.__storage.max_n)}
        self.__n1_right_total = {i: Counter() for i in range(1,self.__storage.max_n)}
        for ngram_size in range(2, self.__storage.max_n+1):
            for ngram in self.__storage[ngram_size]:
                self.__n1_right_count[(ngram_size-1)][ngram[:(ngram_size-1)]] += 1
                self.__n1_right_total[(ngram_size-1)][ngram[:(ngram_size-1)]] += self.__storage[ngram_size][ngram]
                    
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
    
    def __calc_n1_plus(self):
        return sum(1 for v in self.__storage[2].values() if v > 0)
    
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        #if not isinstance(word, str):
        #    raise TypeError('word must be a string!')
        #if not isinstance(context, tuple):
        #    raise TypeError('word must be a string!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        ### YOUR CODE HERE
        if len(context) == 0:
            return max(self.__n1_left[word] - self.__delta, 0.) / self.__n1_plus + self.__delta /self.__voc_size  
                
        m1= max(self.__storage(context + (word, )) - self.__delta, 0.)
        
        
        
        total_count = self.__n1_right_total[len(context)][context]
        total_occurancies = self.__n1_right_count[len(context)][context]
        
        if m1 == 0 and total_occurancies == 0:
            return 0
        
        return m1 / total_count + self.__delta / total_count * total_occurancies * self.__call__(word, context[1:])
        ### END YOUR CODE
        
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [259]:
# Initialize estimator
kn_estimator = KneserNeyProbabilityEstimator(storage)
# Estimating perplexity
print('Simple estimator perplexity = {}'.format(perplexity(kn_estimator, test_sents)))
print(kn_estimator.prob('To be'.split()))
print(kn_estimator.prob('To be or not to be'.split()))

Simple estimator perplexity = 227.203643162
2.65592553964e-06
4.6598344943e-13


## Определение языка документа

**Постановка задачи:**  
Одна из задач, которая может быть решена при помощи языковых моделей $-$ **определение языка документа**. Реализуйте два классификатора для определения языка документа:
1. Наивный классификатор, который будет учитывать частотности символов и выбирать язык текста по признаку: распределение частот символов "наиболее похоже" на распределение частот символов в выбранном языке.
2. Классификатор на основе языковых моделей. Сами придумайте, как он должен работать.  
_Подсказка_: лучше считать n-грамы не по словам, а по символам.

---

**Как представлены данные:**  
Во всех текстовых файлах на каждой строчке записано отдельное предложение.
1. В папочке _data_ находятся две папочки: _full_ и _plain_. В _full_ находятся тексты в той форме, что они были взяты из сети, в _plain_ находятся те же самые тексты, но с них сначала была снята диакритика, а затем русский и греческий тексты были транслитерованы в английский.
2. В каждой из папочек _full_ и _plain_ находятся папочки _train_ и _test_.
3. В _train_ находятся файлы с текстами с говорящими именами, например, _ru.txt_, _en.txt_.
4. В _test_ находятся файлы _1.txt_, _2.txt_, $\dots$ в которых хранятся тексты, язык которых нужно определить. В этой же папочке находится файл _ans.csv_, в котором вы можете найти правильные ответ и проверить, насколько хорошо сработали Ваши алгоритмы.

---

**Что нужно сделать:**  
Напишите два своих классификатора (которые описаны в постановке задачи) и получите максимально возможное accuracy на test-сете. Разрешается использовать только _train_ для обучения.

---

**В данном задании мы не предоставляем стартового кода!**

In [117]:
# Строим ngramm-ы на обучающей выборке
storages_train = defaultdict()

filenames_list = []
for (dirpath, dirnames, filenames) in walk("train"):
    filenames_list.extend(filenames)
    break
    
for filename in filenames_list:
    storages_train[filename] = NGramStorage(ReadWords("train\\" + filename), 3)

In [181]:
# Write the code and estimate accuracy of your method.
# Create your own classifiers.
# Hard code is NOT ALLOWED!

### YOUE CODE HERE

def ReadWords(filename):
    result = []
    with open(filename,'r') as f:
        for line in f:
            for word in line.split():
                result.append(list(word))
    return result

def ReadTrueAnswers(filename):
    result = defaultdict()
    with open(filename,'r') as f:
        for line in f:
            key, value = line.split(',')
            result[key] = value
    return result

def PredictLanguageDistribution(storages_train, storage_test):
    test_size = storage_test[0][()]
    result_language = ''
    best_prob = 9999999
    for key, value in storages_train.items():
        storage_train = storages_train[key]
        storage_train_size = storage_train[0][()]
        prob = 0
        for word in storage_train[1]:
            test_prob = 1. * storage_test[1][word] / test_size
            if test_prob == 0:
                continue
            train_prob = 1. * storage_train[1][word] / storage_train_size
            prob += 1. * train_prob * math.log(train_prob / test_prob)
                
        if prob < best_prob:
            prob = best_prob
            result_language = key
    return result_language



def PredictLanguagePerplexity(storages_train, test_sents):
    result_language = ''
    best_perplexity = 9999999
    for key, value in storages_train.items():
        laplace_estimator = LaplaceProbabilityEstimator(storages_train[key], 0.0001)
        current_perplexity = perplexity(laplace_estimator, test_sents)
        if current_perplexity < best_perplexity:
            best_perplexity = current_perplexity
            result_language = key
    return result_language
### END YOUR CODE


In [101]:
test_filenames_list = []
for (dirpath, dirnames, filenames) in walk("test"):
    test_filenames_list.extend(filenames)
    break

In [148]:
# Выбираем язык файла, у которого наиболее близкое распределение частоты
lang_pred = defaultdict()
for filename in test_filenames_list:
    words = ReadWords("test\\" + filename)
    storage = NGramStorage(words, 1)
    lang_pred[filename] = PredictLanguageDistribution(storages_train, storage)

In [175]:
lang_true = ReadTrueAnswers("ans.csv")
label_true = []
label_pred = []
for key, value_pred in lang_pred.items():
    label_pred.append(lang_pred[key][:2])
    label_true.append(lang_true[key.split('.')[0]][:2])
print "accurancy=", sklearn.metrics.accuracy_score(label_true, label_pred)

accurancy= 0.95


In [182]:
# Выбираем язык файла, у которого наименьшая перплексия
lang_pred = defaultdict()
for filename in test_filenames_list:
    lang_pred[filename] = PredictLanguagePerplexity(storages_train, ReadWords("test\\" + filename))

In [183]:
lang_true = ReadTrueAnswers("ans.csv")
label_true = []
label_pred = []
for key, value_pred in lang_pred.items():
    label_pred.append(lang_pred[key][:2])
    label_true.append(lang_true[key.split('.')[0]][:2])
print "accurancy=", sklearn.metrics.accuracy_score(label_true, label_pred)

accurancy= 1.0


Получили точность по сравнению частот 95%, а по сравнению с помощью ngramm (Использовались триграммы со сглаживанием лапласса) точность 100%.